In [37]:
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
from datetime import date, timedelta
import datetime
from binance.client import Client
# import time
# import math
# import binance
# import plotly as plt
# from matplotlib import pyplot as plt
# import vectorbt as vb

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None 

In [34]:
global api 
api = ""
global private 
private = ""

In [90]:
start_str= "2021-07-03"
client = Client(api,private)
bars = client.get_historical_klines(symbol='BTCUSDT',interval="12h",start_str=start_str)
test_df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
test_df["date"]=pd.to_datetime(test_df["timestamp"],unit="ms").astype(str)
test_df = test_df.drop(["close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
test_df[["open","high","low","close","volume"]] =test_df[["open","high","low","close","volume"]].apply(pd.to_numeric, errors='coerce', axis=1)

smooth = 2
short_period = 10
long_period = 100

test_df["short EMA"] = test_df["close"].copy()
for i in range(1, len(test_df)):
    test_df["short EMA"][i] = test_df["short EMA"][i - 1] * (1 - smooth / (short_period + 1)) + test_df["short EMA"][i] * smooth / (short_period + 1)

test_df["short DEMA"] = test_df["short EMA"].copy()

test_df["long EMA"] = test_df["close"].copy()
for i in range(1, len(test_df)):
    test_df["long EMA"][i] = test_df["long EMA"][i - 1] * (1 - smooth / (long_period + 1)) + test_df["long EMA"][i] * smooth / (long_period + 1)

test_df["long DEMA"] = test_df["long EMA"].copy()

In [91]:
test_df["boo"] = (test_df["short DEMA"] - test_df["long DEMA"] > 0)
ind = test_df[["timestamp","boo"]]

In [103]:
def backtesting(indicator,symbol,startdate,enddate=None,timeframe = "1d",capital=1000,storage=0,delay=0):

	if delay !=0 :
		st = datetime.datetime.strptime(startdate,'%Y-%m-%d').date()-timedelta(delay)
		startdate = st.strftime("%Y-%m-%d")
	client = Client(api,private)
	bars = client.get_historical_klines(symbol='BTCUSDT',interval=timeframe,start_str=startdate,end_str= enddate)
	df = pd.DataFrame(bars[:],columns=["timestamp","open","high","low","close","volume", "close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"])
	df = df.drop(["close_time", "quote_asset_volume", "number_of_trade", "TBB", "TBQ", "ignore"], axis=1)
	df["date"]=pd.to_datetime(df["timestamp"],unit="ms").astype(str)
	df[["open","high","low","close","volume"]] =df[["open","high","low","close","volume"]].apply(pd.to_numeric, errors='coerce', axis=1)
	
	for i in range(0, len(df)):
		if indicator["timestamp"][0] == df["timestamp"][i]:
			st = i
			break
	else:
		return "error in date"
	money = capital
	mink = min(len(indicator)-2,len(df)-st)
	for i,j in zip(range(150, 2 + mink),range(st+150, st + mink)):
	# for i,j in zip(range(2, 2 + mink),range(st, st + mink)):
		if (indicator.iat[i-2,1]==0 and indicator.iat[i-1,1]==1):
			if(storage<0):
				money += storage * df["close"][j]
				storage = 0
			storage += money / df["close"][j]
			money = 0

		elif (indicator.iat[i-2,1]==1 and indicator.iat[i-1,1]==0):
			if (storage > 0):
				money += storage * df["close"][j]
				storage = 0
			storage -= money / df["close"][j]
			money -= storage * df["close"][j]


	profit =(money + storage * df["close"][len(df)-1])
	
	returns = [symbol, profit, profit / capital]
	return(returns)

backtesting(ind,"BTCUSDT","2021-07-03",timeframe="12h")


['BTCUSDT', 1103.1533552201997, 1.1031533552201997]